<a href="https://colab.research.google.com/github/JLearning5/Credit-Card-Fraud-Detection-project/blob/main/notebooks/Fraud_Detection_preview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
# Install gdown to download files from Google Drive
!pip install gdown

# Download the dataset from Google Drive using the file ID
import gdown

file_id = '1cIYhJzufKd1R9t3qQtwYnLbjvwzybqq4'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'dataset.csv'
gdown.download(url, output, quiet=False)

# Load the dataset
import pandas as pd

df = pd.read_csv('dataset.csv')

# Display the first few rows of the dataset
df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1cIYhJzufKd1R9t3qQtwYnLbjvwzybqq4
From (redirected): https://drive.google.com/uc?id=1cIYhJzufKd1R9t3qQtwYnLbjvwzybqq4&confirm=t&uuid=d05f58be-e9c7-4550-90ab-e9e061014832
To: /content/dataset.csv
100%|██████████| 151M/151M [00:04<00:00, 36.0MB/s]


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv(r'/content/dataset.csv')

legit = df[df.Class == 0]
fraud = df[df.Class == 1]

# Separate features and target variable
X = df.drop('Class', axis=1)
y = df['Class']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply SMOTE to the training data
sampling = SMOTE()
X_train_resampled, y_train_resampled  = sampling.fit_resample(X_train, y_train)
X_test_resampled, y_test_resampled =  sampling.fit_resample(X_test, y_test)

# Train a logistic regression model on the resampled data
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred = model.predict(X_test_resampled)
y_pred_proba = model.predict_proba(X_test_resampled)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test_resampled, y_pred)

# Suppress specific sklearn warnings and replace them with custom messages
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# Function to handle predictions
def predict_transaction(features):
    try:
        # Convert features to numpy array
        features = np.array([float(i) for i in features.split(',')]).reshape(1, -1)

        # Scale the features
        features_scaled = scaler.transform(features)

        # Predict the class
        prediction = model.predict(features_scaled)
        prediction_proba = model.predict_proba(features_scaled)

        result = 'Fraudulent' if prediction[0] == 1 else 'Legitimate'
        confidence = prediction_proba[0][prediction[0]]

        return f"Prediction: {result}<br>Confidence: {confidence:.2f}"
    except Exception as e:
        return f"Error: {e}"

# Generate the classification report as a dictionary
report_dict = classification_report(y_test_resampled, y_pred, output_dict=True)

# Extract and format the accuracy value to 11 decimal places
accuracy = report_dict['accuracy']
formatted_accuracy = f"{accuracy:.11f}"

# Function to convert classification report dictionary to an HTML table with colored cells
def classification_report_to_html(report):
    def get_color(value):
        """Get color based on value for heatmap effect."""
        if value > 100:
            color = f"rgba(135, 206, 250, {value})"  # Light Blue
        elif value >= 0.75:
            color = f"rgba(255, 255, 0, {value})"  # Yellow
        elif value >= 0.5:
            color = f"rgba(0, 255, 0, {value})"  # Green
        else:
            color = f"rgba(255, 0, 0, {value})"  # Red
        return color

    html = "<center><table style='width:1200px; color:black; background-color:white; border-collapse: collapse;'</center>"
    html += "<thead><tr style='border:1px solid black;'>"
    html += "<th style='border:1px #333; background-color:#333;'></th>"  # Empty header cell with black background
    for col in report['0'].keys():
        html += f"<th style='border:1px solid black; background-color:white;'>{col}</th>"
    html += "</tr></thead><tbody>"
    for label, metrics in report.items():
        html += f"<tr style='border:1px solid black;'><td style='border:1px solid black; background-color:white;'>{label}</td>"
        if isinstance(metrics, dict):
            for key, value in metrics.items():
                color = get_color(value)
                html += f"<td style='border:1px solid black; background-color:{color};'>{value:.2f}</td>"
        else:
            html += f"<td style='border:1px solid black;' colspan='{len(report['0'])}'>{metrics:.2f}</td>"
        html += "</tr>"
    html += "</tbody></table>"
    return html

# Create the text box widget with increased width and height
text_box = widgets.Textarea(
    value='',
    placeholder='Enter features here... e.g., 0,-1.359807134,-0.072781173,2.536346738,1.378155224,-0.33832077,0.462387778,0.239598554,0.098697901,0.36378697,0.090794172,-0.551599533,-0.617800856,-0.991389847,-0.311169354,1.468176972,-0.470400525,0.207971242,0.02579058,0.40399296,0.251412098,-0.018306778,0.277837576,-0.11047391,0.066928075,0.128539358,-0.189114844,0.133558377,-0.021053053,149.62',
    description='Features:',
    disabled=False,
    layout=widgets.Layout(width='1200px', height='75px', background_color='#333', color='white')
)

# Create the button widget with vertical alignment
clear_button = widgets.Button(
    description='Clear Data',
    disabled=False,
    button_style='danger',
    tooltip='Click to clear',
    icon='trash',
    layout=widgets.Layout(margin='20px 30px 30px 60px', background_color='#333', color='white')
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',
    tooltip='Click to submit',
    icon='check',
    layout=widgets.Layout(margin='20px 30px 30px 100px', background_color='#333', color='white')
)

output = widgets.Output()

def on_clear_button_click(b):
    text_box.value = ''

def on_submit_button_click(b):
    with output:
        output.clear_output()
        result = predict_transaction(text_box.value)
        # Display the result with white-colored text
        display(HTML(f"<span style='color:white;'><center>{result}</center></span>"))

class_report_html = classification_report_to_html(report_dict)

clear_button.on_click(on_clear_button_click)
submit_button.on_click(on_submit_button_click)

# Create the header using HTML widget
header1 = widgets.HTML("<h1 style='color:white; text-align:center;'>Credit Card Fraud Detection Using Logistic Regression Model</h1>")
header2 = widgets.HTML(f"<p style='color:white; text-align:center;'><b><i>Model Accuracy Score</i></b>: {formatted_accuracy}</p>")
header3 = widgets.HTML("<h3 style='color:white; text-align:center;'>Enter features to check if the transaction is legitimate or fraudulent</h3>")
header4 = widgets.HTML(f"<h3 style='color:white; text-align:center;'>Classification Report</h3>{class_report_html}")

details_header1 = widgets.HTML("<h2 style='color:white; text-align:center; margin-bottom: -8px;'>Credit Card Fraud Detection Project</h2>")
details_header2 = widgets.HTML("<p style='color:white; text-align:center;'>This project aims to detect fraudulent credit card transactions using a Logistic Regression model. The dataset used is the 'Credit Card Fraud Detection' dataset from Kaggle.</p>")

details_header3 = widgets.HTML("""
    <p style='text-align:center;'>
        <span style='color:white;'><b><i>Source: </i></b></span>
        <a href='https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data' target='_blank' style='color:#ADD8E6;'>
            https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data
        </a>
    </p>
""")
details_header4 = widgets.HTML("<h2 style='color:white; text-align:center;'>Project Overview</h2>")
details_header5 = widgets.HTML("""
    <p style='color:white; text-align:center; max-width:1000px; margin:auto;'>
        The goal of this project is to build a machine learning model to detect fraudulent transactions.
        The dataset contains transactions made by credit cards in September 2013 by European cardholders.
        The dataset is highly imbalanced, with only 0.172% of transactions being fraudulent.
    </p>
""")
details_header6 = widgets.HTML("<h2 style='color:white; text-align:center; margin-bottom: -8px;'>Features</h2>")
details_header7a = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'><b><i>Data Preprocessing: </i></b>Handling imbalanced data using over-sampling.</p>")
details_header7b = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'><b><i>Model Training: </i></b>Logistic Regression model for classification.</p>")
details_header7c = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'><b><i>Evaluation: </i></b>Model evaluation using accuracy score, ROC curve, and precision-recall curve.</p>")
details_header7d = widgets.HTML("<p style='color:white; text-align:center;'><b><i>Visualisation: </i></b>Various plots to visualise class distribution, transaction amounts, time distribution, correlation matrix, and more.</p>")
details_header8 = widgets.HTML("<h2 style='color:white; text-align:center; margin-bottom: -8px;'>Dependencies</h2>")
details_header9 = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'><b><i>The project uses the following libraries:</i></b></p>")
details_header9a = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'>Python 3.9</p>")
details_header9b = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'>NumPy</p>")
details_header9c = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'>Pandas</p>")
details_header9d = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'>Seaborn</p>")
details_header9e = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'>Matplotlib</p>")
details_header9f = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'>Scikit-learn</p>")
details_header9g = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'>Ipywidgets</p>")
details_header9h = widgets.HTML("<p style='color:white; text-align:center; margin-bottom: -8px;'>Imbalanced-learn</p>")

header5 = widgets.HTML(f"<h3 style='color:white; text-align:center; margin-bottom: -8px;'>Class Distribution</h3>")
header6 = widgets.HTML(f"<p style='color:white; text-align:center;'>Visualising the count of each class using bar plot</p>")
header7 = widgets.HTML(f"<h3 style='color:white; text-align:center; margin-bottom: -8px;'>Amount Distribution</h3>")
header8 = widgets.HTML(f"<p style='color:white; text-align:center;'>Visualising the transaction amounts using histogram and box Plot</p>")
header9 = widgets.HTML(f"<h3 style='color:white; text-align:center; margin-bottom: -8px;'>Time Distribution</h3>")
header10 = widgets.HTML(f"<p style='color:white; text-align:center;'>Visualising the time of transactions using histogram and box plot</p>")
header11 = widgets.HTML(f"<h3 style='color:white; text-align:center; margin-bottom: -8px;'>Correlation Matrix</h3>")
header12 = widgets.HTML(f"<p style='color:white; text-align:center;'>Visualising the correlation matrix using heatmap</p>")
header13 = widgets.HTML(f"<h3 style='color:white; text-align:center; margin-bottom: -8px;'>Pairplot</h3>")
header14 = widgets.HTML(f"<p style='color:white; text-align:center; margin-bottom: -8px'>Visualising the relationships between features using pairplot</p>")
header15 = widgets.HTML(f"<p style='color:white; text-align:center; margin-bottom: -8px'><b><i>before sampling</i></b></p>")
header16 = widgets.HTML(f"<p style='color:white; text-align:center; margin-bottom: -8px'><b><i>after sampling</i></b></p>")
header17 = widgets.HTML(f"<h3 style='color:white; text-align:center; margin-bottom: -8px;'>Receiver Operating Characteristic (ROC) Curve</h3>")
header18 = widgets.HTML(f"<p style='color:white; text-align:center;'>Visualising the ROC curve for the model</p>")
header19 = widgets.HTML(f"<h3 style='color:white; text-align:center; margin-bottom: -8px;'>Precision-Recall Curve</h3>")
header20 = widgets.HTML(f"<p style='color:white; text-align:center;'>Visualising the precision-recall curve for the model</p>")
header21 = widgets.HTML(f"<h3 style='color:white; text-align:center; margin-bottom: -8px;'>Confusion Matrix</h3>")
header22 = widgets.HTML(f"<p style='color:white; text-align:center;'>Visualising the confusion matrix for the model</p>")

# Generate and save the plot as an image
plt.figure(figsize=(10, 6))
sns.countplot(x='Class', data=df)
plt.title('Class Distribution before sampling')
plot_file_before = 'class_distribution_before.png'
plt.savefig(plot_file_before)
plt.close()

plt.figure(figsize=(10, 6))
sns.countplot(x=y_train_resampled)
plt.title('Class Distribution after Sampling')
plt.xlabel('Class')
plt.ylabel('Count')
plot_file_after = 'class_distribution_after.png'
plt.savefig(plot_file_after)
plt.close()

# Display the saved images using Image widgets
image_widget_before = widgets.Image(
    value=open(plot_file_before, 'rb').read(),
    format='png',
    width=600,
    height=400
)

image_widget_after = widgets.Image(
    value=open(plot_file_after, 'rb').read(),
    format='png',
    width=600,
    height=400
)

# Generate and save the amount distribution plots - Histogram
plt.figure(figsize=(10, 6))
sns.histplot(df['Amount'], bins=50, kde=True)
plt.title('Amount Distribution in Original Dataset')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plot_file_amount_before_histogram = 'amount_distribution_before_Histogram.png'
plt.savefig(plot_file_amount_before_histogram)
plt.close()

# Plot the histogram for the 'Amount' distribution in the resampled dataset
# Create a DataFrame from the resampled data for easier plotting
resampled_df = pd.DataFrame(X_train_resampled, columns=X.columns)
resampled_df['Class'] = y_train_resampled

plt.figure(figsize=(10, 6))
sns.histplot(resampled_df['Amount'], bins=50, kde=True)
plt.title('Amount Distribution in Resampled Dataset')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plot_file_amount_after_histogram = 'amount_distribution_after_Histogram.png'
plt.savefig(plot_file_amount_after_histogram)
plt.close()

# Generate and save the amount distribution plots - Box Plot
plt.figure(figsize=(10, 6))
sns.boxplot(x='Class', y='Amount', data=df)
plt.title('Amount Distribution in Original Dataset')
plt.yscale('log')  # Use log scale if there are large outliers
plot_file_amount_before_BoxPlot = 'amount_distribution_after_BoxPlot.png'
plt.savefig(plot_file_amount_before_BoxPlot)
plt.close()

resampled_df = pd.DataFrame(X_train_resampled, columns=X.columns)
resampled_df['Class'] = y_train_resampled

plt.figure(figsize=(10, 6))
sns.boxplot(x='Class', y='Amount', data=resampled_df)
plt.title('Amount Distribution in Resampled Dataset')
plt.yscale('log')  # Use log scale if there are large outliers
plot_file_amount_after_BoxPlot = 'amount_distribution_after_BoxPlot.png'
plt.savefig(plot_file_amount_after_BoxPlot)
plt.close()

# Display the saved images using Image widgets
image_widget_amount_histogram_before = widgets.Image(
    value=open(plot_file_amount_before_histogram, 'rb').read(),
    format='png',
    width=600,
    height=400
)

image_widget_amount_histogram_after = widgets.Image(
    value=open(plot_file_amount_after_histogram, 'rb').read(),
    format='png',
    width=600,
    height=400
)

image_widget_amount_BoxPlot_before = widgets.Image(
    value=open(plot_file_amount_before_BoxPlot, 'rb').read(),
    format='png',
    width=600,
    height=400
)

image_widget_amount_BoxPlot_after = widgets.Image(
    value=open(plot_file_amount_after_BoxPlot, 'rb').read(),
    format='png',
    width=600,
    height=400
)

# Generate and save the time distribution plots - histogram
plt.figure(figsize=(10, 6))
sns.histplot(df['Time'], bins=50, kde=True)
plt.title('Time Distribution in Original Dataset')
plt.xlabel('Time')
plt.ylabel('Frequency')
plot_file_time_before_histogram = 'time_distribution_before_histogram.png'
plt.savefig(plot_file_time_before_histogram)
plt.close()

# Plot the histogram for the 'Time' distribution in the resampled dataset
# Create a DataFrame from the resampled data for easier plotting
resampled_df = pd.DataFrame(X_train_resampled, columns=X.columns)
resampled_df['Class'] = y_train_resampled

plt.figure(figsize=(10, 6))
sns.histplot(resampled_df['Time'], bins=50, kde=True)
plt.title('Time Distribution in Resampled Dataset')
plt.xlabel('Time')
plt.ylabel('Frequency')
plot_file_time_after_histogram = 'time_distribution_after_histogram.png'
plt.savefig(plot_file_time_after_histogram)
plt.close()

# Generate and save the time distribution plots - box plot
plt.figure(figsize=(10, 6))
sns.boxplot(x='Class', y='Time', data=df)
plt.title('Time Distribution in Original Dataset')
plt.yscale('log')  # Use log scale if there are large outliers
plot_file_time_before_BoxPlot = 'time_distribution_before_BoxPlot.png'
plt.savefig(plot_file_time_before_BoxPlot)
plt.close()

# Plot the box plot for the Time feature in the resampled dataset
resampled_df = pd.DataFrame(X_train_resampled, columns=X.columns)
resampled_df['Class'] = y_train_resampled

plt.figure(figsize=(10, 6))
sns.boxplot(x='Class', y='Time', data=resampled_df)
plt.title('Time Distribution in Resampled Dataset')
plt.yscale('log')  # Use log scale if there are large outliers
plot_file_time_after_BoxPlot = 'time_distribution_after_BoxPlot.png'
plt.savefig(plot_file_time_after_BoxPlot)
plt.close()

# Display the saved images using Image widgets
image_widget_time_histogram_before = widgets.Image(
    value=open(plot_file_time_before_histogram, 'rb').read(),
    format='png',
    width=600,
    height=400
)

image_widget_time_histogram_after = widgets.Image(
    value=open(plot_file_time_after_histogram, 'rb').read(),
    format='png',
    width=600,
    height=400
)

image_widget_time_BoxPlot_before = widgets.Image(
    value=open(plot_file_time_before_BoxPlot, 'rb').read(),
    format='png',
    width=600,
    height=400
)

image_widget_time_BoxPlot_after = widgets.Image(
    value=open(plot_file_time_after_BoxPlot, 'rb').read(),
    format='png',
    width=600,
    height=400
)

# Generate and save the correlation matrix plots
plt.figure(figsize=(10, 6))
corr_matrix_original = df.corr()
sns.heatmap(corr_matrix_original, annot=True, fmt='.2f', cmap='coolwarm', annot_kws={"size": 6})
plt.title('Correlation Matrix for Original Dataset', fontsize=12)
plot_file_correlation_matrix_before = 'correlation_matrix_before.png'
plt.savefig(plot_file_correlation_matrix_before, dpi=300)
plt.close()

# Convert the resampled training data back into a DataFrame
X_train_resampled_df = pd.DataFrame(X_train_resampled, columns=X.columns)

# Add the target variable to the resampled DataFrame
X_train_resampled_df['Class'] = y_train_resampled.values

# Compute the correlation matrix
corr_matrix = X_train_resampled_df.corr()

plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', annot_kws={"size": 6})
plt.title('Correlation Matrix for Over-Sampled Dataset', fontsize=12)
plot_file_correlation_matrix_after = 'correlation_matrix_after.png'
plt.savefig(plot_file_correlation_matrix_after, dpi=300)
plt.close()


# Display the saved images using Image widgets
image_widget_correlation_matrix_before = widgets.Image(
    value=open(plot_file_correlation_matrix_before, 'rb').read(),
    format='png',
    width=1200,
    height=400
)

image_widget_correlation_matrix_after = widgets.Image(
    value=open(plot_file_correlation_matrix_after, 'rb').read(),
    format='png',
    width=1200,
    height=400
)

# Select specific columns for the pairplot
selected_columns = ['V1', 'V2', 'V27', 'V28', 'Amount', 'Class']

# Generate and save the Pairplot
plt.figure(figsize=(10, 6))
df_sampled = df[selected_columns].sample(frac=0.1, random_state=42)  # Sample a fraction of the original dataset for visualization
sns.pairplot(df_sampled, hue='Class', palette='Set1')
plt.title('Pairplot of Original Dataset')
plot_file_pairplot_before = 'pairplot_before.png'
plt.savefig(plot_file_pairplot_before)
plt.close()

# Prepare the resampled dataset for pairplot
X_train_resampled_df = pd.DataFrame(X_train_resampled, columns=X.columns)
X_train_resampled_selected = X_train_resampled_df[selected_columns[:-1]]
y_train_resampled_df = pd.Series(y_train_resampled, name='Class')
df_resampled = pd.concat([X_train_resampled_selected, y_train_resampled_df], axis=1)

# Sample a fraction of the resampled dataset for visualization
df_resampled_sampled = df_resampled.sample(frac=0.1, random_state=42)

plt.figure(figsize=(10, 6))
sns.pairplot(df_resampled_sampled, hue='Class', palette='Set1')
plt.title('Pairplot of Resampled Dataset')
plot_file_pairplot_after = 'pairplot_after.png'
plt.savefig(plot_file_pairplot_after)
plt.close()

# Display the saved images using Image widgets
image_widget_pairplot_before = widgets.Image(
    value=open(plot_file_pairplot_before, 'rb').read(),
    format='png',
    width=1200,
    height=400
)

image_widget_pairplot_after = widgets.Image(
    value=open(plot_file_pairplot_after, 'rb').read(),
    format='png',
    width=1200,
    height=400
)

# Generate and save the ROC Curve
# Generate a ROC curve
fpr, tpr, thresholds = roc_curve(y_test_resampled, y_pred_proba)
roc_auc = roc_auc_score(y_test_resampled, y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plot_file_ROC_Curve = 'ROC_Curve.png'
plt.savefig(plot_file_ROC_Curve)
plt.close()

# Display the saved images using Image widgets
image_widget_ROC_Curve = widgets.Image(
    value=open(plot_file_ROC_Curve, 'rb').read(),
    format='png',
    width=1200,
    height=400
)

# Generate and save the Precision-Recall Curve
# Generate a precision-recall curve for the resampled dataset
precision, recall, thresholds = precision_recall_curve(y_test_resampled, y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(recall, precision, marker='.', label='Logistic Regression')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid()
plot_file_precision_recall_curve = 'precision_recall_curve.png'
plt.savefig(plot_file_precision_recall_curve)
plt.close()

# Display the saved images using Image widgets
image_widget_precision_recall_curve = widgets.Image(
    value=open(plot_file_precision_recall_curve, 'rb').read(),
    format='png',
    width=1200,
    height=400
)

# Generate and save the Confusion Matrix
conf_matrix = confusion_matrix(y_test_resampled, y_pred)
cm_display = ConfusionMatrixDisplay(conf_matrix, display_labels=model.classes_)

plt.figure(figsize=(8, 6))
cm_display.plot()
plt.title("Confusion Matrix for Resampled Dataset")
plot_file_confusion_matrix = 'confusion_matrix.png'
plt.savefig(plot_file_confusion_matrix)
plt.close()

# Display the saved images using Image widgets
image_widget_confusion_matrix = widgets.Image(
    value=open(plot_file_confusion_matrix, 'rb').read(),
    format='png',
    width=1200,
    height=400
)

# Create the VBox and HBox containers
text_box_box = widgets.VBox([header3, text_box])
button_box = widgets.HBox([submit_button, clear_button], layout=widgets.Layout(justify_content='center'))
image_widget_box_before_after = widgets.HBox([image_widget_before, image_widget_after])
image_widget_amount_histogram_box = widgets.HBox([image_widget_amount_histogram_before, image_widget_amount_histogram_after])
image_widget_amount_boxplot_box = widgets.HBox([image_widget_amount_BoxPlot_before, image_widget_amount_BoxPlot_after])
image_widget_time_histogram_box = widgets.HBox([image_widget_time_histogram_before, image_widget_time_histogram_after])
image_widget_time_boxplot_box = widgets.HBox([image_widget_time_BoxPlot_before, image_widget_time_BoxPlot_after])
image_widget_correlation_matrix_box_before = widgets.HBox([image_widget_correlation_matrix_before])
image_widget_correlation_matrix_box_after = widgets.HBox([image_widget_correlation_matrix_after])
image_widget_pairplot_box_before = widgets.HBox([image_widget_pairplot_before])
image_widget_pairplot_box_after = widgets.HBox([image_widget_pairplot_after])
image_widget_roc_curve_box = widgets.HBox([image_widget_ROC_Curve])
image_widget_precision_recall_curve_box = widgets.HBox([image_widget_precision_recall_curve])
image_widget_confusion_matrix_box = widgets.HBox([image_widget_confusion_matrix])

# Create the container box with centered content
container = widgets.Box(
    [
        widgets.VBox(
            [
                header1,
                header2,
                text_box_box,
                button_box,
                output,
                header4,
                details_header1,
                details_header2,
                details_header3,
                details_header4,
                details_header5,
                details_header6,
                details_header7a,
                details_header7b,
                details_header7c,
                details_header7d,
                details_header8,
                details_header9,
                details_header9a,
                details_header9b,
                details_header9c,
                details_header9d,
                details_header9e,
                details_header9f,
                details_header9g,
                details_header9h,
                header5,
                header6,
                image_widget_box_before_after,
                header7,
                header8,
                image_widget_amount_histogram_box,
                image_widget_amount_boxplot_box,
                header9,
                header10,
                image_widget_time_histogram_box,
                image_widget_time_boxplot_box,
                header11,
                header12,
                image_widget_correlation_matrix_box_before,
                image_widget_correlation_matrix_box_after,
                header13,
                header14,
                header15,
                image_widget_pairplot_box_before,
                header16,
                image_widget_pairplot_box_after,
                header17,
                header18,
                image_widget_roc_curve_box,
                header19,
                header20,
                image_widget_precision_recall_curve_box,
                header21,
                header22,
                image_widget_confusion_matrix_box
            ],
            layout=widgets.Layout(
                display='flex',
                align_items='center',
                justify_content='center',
                width='100%',
                flex_direction='column'
            )
        )
    ],
    layout=widgets.Layout(
        display='flex',
        align_items='center',
        justify_content='center',
        width='100%',
        flex_direction='column'
    )
)

# Apply custom CSS styling for dark theme
custom_css = """
<style>
    .widget-label {
        color: white !important;
    }
    .widget-html-content {
        background-color: #333 !important;
        color: white !important;
    }
    .widget-output {
        background-color: #333 !important;
        color: white !important;
    }
    .widget-box {
        background-color: #333 !important;
    }
    table {
        width: 100%;
        border-collapse: collapse;
        color: white;
    }
    th, td {
        border: 1px solid white;
        padding: 8px;
        text-align: left;
    }
    th {
        background-color: #444;
    }
</style>
"""

display(HTML(custom_css))
display(container)

Box(children=(VBox(children=(HTML(value="<h1 style='color:white; text-align:center;'>Credit Card Fraud Detecti…

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>